In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

#Reading dataset and storing it
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print()
        dataframe =pd.read_csv(os.path.join(dirname, filename))


In [2]:
#Deleting unused columns and creating characterists df 
dataframe = dataframe.drop(['name'], axis=1)
df_car = dataframe.drop(['status'], axis=1)

#Separating classes
PD = dataframe[dataframe['status'] == 0]
PD = PD.drop(['status'], axis=1)
C1 = PD.values
C1_f, C1_c = C1.shape

HC = dataframe[dataframe['status'] == 1]
HC = HC.drop(['status'], axis=1)
C2 = HC.values
C2_f, C2_c = C2.shape



In [3]:
matcar = df_car.values #Converting data as a matrix
mat_car = matcar[:, :] # copy matrix information
covmatrix = np.cov(mat_car, rowvar=False) #Covariance Matriz
mat_cinv = np.linalg.inv(covmatrix)# Inverse Matrix

In [4]:
peg=np.inf #Minimum error percentage 
for car in range(1, 22):
    for k in range(3, 15):
        matcinv=mat_cinv[:car,:car]
        mat_car = matcar[:, :car]
        #Initializing errors per classes
        e1 = 0; e2 = 0
        # 33.33% of samples for training
        lim_C1 = int(np.round(C1_f * 0.333))
        lim_C2 = int(np.round(C2_f * 0.333))
        train_sam1 = C1[:lim_C1 - 1, :car]
        train_sam2 = C2[:lim_C2 - 1, :car]
        train_sam = np.concatenate((train_sam1, train_sam2), axis=0)
        
        #All samples
        samples = np.concatenate((C1[lim_C1:C1_f, :car], C2[lim_C2:C2_f, :car]), axis=0)

        train_r, _ = train_sam.shape
        samples_r, _ = samples.shape

        for i in range(samples_r):
            cont1 = 0
            cont2 = 0
            X = samples[i, :car]
            neig = np.zeros(train_r)

            #Computes the distance between them
            for j in range(train_r):
                z1 = train_sam[j, :]
                neig[j] = np.dot(np.dot((X - z1).T, matcinv), (X - z1))

            # Find the maximums
            for g in range(k):
                pos = np.argmin(neig)
                if pos < C1_f:
                    cont1 += 1
                else:
                    cont2 += 1
                neig[pos] = np.min(neig) + np.max(neig)

            cont = [cont1, cont2]
            class_ = np.argmax(cont)

            if i < (samples_r / 2 + 1):
                if cont1 == cont2 or class_ != 0:
                    e1 += 1
            else:
                if cont1 == cont2 or class_ != 1:
                    e2 += 1

        pe1 = e1 / C1_f * 100
        pe2 = e2 / C2_f * 100
        pe = (e1 + e2) / (C1_f + C2_f) * 100
        if pe<peg:
            peg=pe
            print("\n")
            print('For {} characteristics and k= {}'.format(int(car)+1, k+1))
            print("PD class Error: {:.2f} percentage".format(pe1))
            print("HC class Error: {:.2f} percentage".format(pe2))
            print("Global Error: {:.2f} percentage".format(pe))
        



For 2 characteristics and k= 4
PD class Error: 20.83 percentage
HC class Error: 36.73 percentage
Global Error: 32.82 percentage


For 2 characteristics and k= 6
PD class Error: 16.67 percentage
HC class Error: 35.37 percentage
Global Error: 30.77 percentage


For 4 characteristics and k= 4
PD class Error: 27.08 percentage
HC class Error: 31.29 percentage
Global Error: 30.26 percentage


For 6 characteristics and k= 10
PD class Error: 2.08 percentage
HC class Error: 38.78 percentage
Global Error: 29.74 percentage


For 7 characteristics and k= 6
PD class Error: 29.17 percentage
HC class Error: 29.25 percentage
Global Error: 29.23 percentage


For 9 characteristics and k= 4
PD class Error: 12.50 percentage
HC class Error: 34.01 percentage
Global Error: 28.72 percentage
